# Lens MTF Model

In this tutorial, we will show how to use prysm to model the MTF of a lens based on a polynomial model of its aberrations.  We will utilize the concepts from the [First Diffraction Model](./First-Diffraction-Model.ipynb) tutorial in constructing the forward model.

MTF is defined as the magnitude of the Fourier transform of the Point Spread Function (PSF), normalized by its value at the origin.  Without writing the normalization, that is simply:

$$ \text{MTF}\left(\nu_x,\nu_y\right) = \left| \mathfrak{F}\left[\text{PSF}\left(x,y\right)\right] \right| $$

To make this tutorial a bit more interesting, we will use an N-sided aperture, as if our lens were stopped down and has a finite number of aperture blades.  We will also assume no vignetting.  Instead of Hopkins' polynomials as used previously, we will use Zernike polynomials which are orthogonal over the unit disk.  Everything scales with F/#, but we'll assume its 8 and the focal length is 50 mm as reasonable photographic examples.

In [ ]:
%load_ext autoreload
%autoreload 2

from prysm.coordinates import make_xy_grid, cart_to_polar
from prysm.geometry import regular_polygon

from prysm.polynomials import zernike_nm

from matplotlib import pyplot as plt

In [ ]:
efl = 50
fno = 8

x, y = make_xy_grid(256, diameter=efl/fno)
dx = x[0,1]-x[0,0]
r, t = cart_to_polar(x, y)
radius = efl/fno/2
rho = r / radius

aperture = regular_polygon(7, radius, x, y)

plt.imshow(aperture, origin='lower')

We will assume for the moment that the illumination is monochromatic, as a separate tutorial deals with polychromatic propagation.  We will also assume the correction of the lens is so-so at its maximum aperture of F/1.4, and decide somewhat arbitrarily that it improves by 20% for each stop the F/# is reduced.  F/1.4 to F/8 is 5 stops, so we have 1.2^5 reduction in wavefront error.  There is no physical basis for these assumptions, but they are being made by the user, not the library.

In [ ]:
from prysm.propagation import Wavefront
wvl = 0.55 # mid visible band, um

full_aperture_opd = wvl*0.75*1e3 # nm, 3/4 of a wave, 1e3 = um to nm
reduced_opd = full_aperture_opd / (1.5**5)
mode = zernike_nm(4, 0, rho, t)
opd = mode * reduced_opd
pup = Wavefront.from_amp_and_phase(aperture, opd, wvl, dx)
coherent_psf = pup.focus(efl, Q=2)

At this point, we are in posession of the coherent PSF, which we will recall can be converted to the incoherent PSF with the `.intensity` computed property.  From there, we simply use the `mtf_from_psf` function to compute the MTF.

In [ ]:
from prysm.otf import mtf_from_psf, diffraction_limited_mtf
psf = coherent_psf.intensity
mtf = mtf_from_psf(psf, psf.dx)

This is the diffraction limited MTF for a circular aperture, but it's close enough for the septagon example.

We can start by plotting the X and Y slices of the MTF.  If we are on axis, or aligned to a cartesian axis of the image plane, these are the tangential and sagittal MTFs.

In [ ]:
fig, ax = mtf.slices().plot(['x', 'y', 'azavg'], xlim=(0,200))

ax.plot(fx, difflim, ls=':', c='k', alpha=0.75, zorder=1)
ax.set(xlabel='Spatial frequency, cy/mm', ylabel='MTF')

We can see the lens would be far from diffraction limited for a broad range of frequencies, and that the x and y MTFs are identical.  The latter follows from spherical aberration, $Z_4^0$ being rotationally invariant.  What if the lens had an equivalent amount of coma?

In [ ]:
reduced_opd = full_aperture_opd / (1.5**5)
mode = zernike_nm(3, 1, rho, t)
opd = mode * reduced_opd
pup = Wavefront.from_amp_and_phase(aperture, opd, wvl, dx)
coherent_psf = pup.focus(efl, Q=2)
psf = coherent_psf.intensity
mtf = mtf_from_psf(psf, psf.dx)

fig, ax = mtf.slices().plot(['x', 'y', 'azavg'], xlim=(0,200))

ax.plot(fx, difflim, ls=':', c='k', alpha=0.75, zorder=1)
ax.set(xlabel='Spatial frequency, cy/mm', ylabel='MTF')

The MTF would be a bit higher, but it no longer would rotationally invariant.

If you were interested in the phase transfer function or the OTF itself, the functions are `ptf_from_psf` and `otf_from_psf`, and they work the same way.

In summary, to model the MTF of a system:

- create a model of the pupil

- create a model of the OPD within the pupil

- propagate the pupil to a PSF plane and take its intensity (for incoherent systems)

- use `mtf_from_psf` to compute the MTF